In [9]:
import sys, pathlib, importlib, inspect
import pennylane as qml
from pennylane import numpy as np
import torch
from pennylane.transforms import make_tape
from pennylane.qcut import cut_circuit
from pennylane.qcut.processing import qcut_processing_fn
#I just realized the kernel isn't even pixi's...
# double checking versions
print("Python executable:", sys.executable)
print("Python version   :", sys.version.split()[0])
print("PennyLane        :", qml.__version__)
print("Torch            :", torch.__version__)

PROJECT_ROOT = pathlib.Path.cwd().resolve().parent.parent
SRC = PROJECT_ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.append(str(SRC))

import circuit.quantum_galton_board as qgb
importlib.reload(qgb)

Python executable: c:\Users\13174\Documents\GaltonBoard\.pixi\envs\default\python.exe
Python version   : 3.12.11
PennyLane        : 0.41.0
Torch            : 2.4.0+cpu


<module 'circuit.quantum_galton_board' from 'C:\\Users\\13174\\Documents\\GaltonBoard\\src\\circuit\\quantum_galton_board.py'>

In [11]:

levels = 6
num_wires = 2 * levels
cut_wires = [6]
shots = 20_000

galton = qgb.galton_template(levels, bias=0.3, coherence=False)      # no resets → no measurements

print(type(galton))                        # should be <class 'function'>
print(isinstance(galton, qml.QNode))       # should be False
print("decorators:", inspect.getsource(galton).splitlines()[0])


<class 'function'>
False
decorators:     def template():


In [12]:
#QAOA layer on wires 8-15 to cut the circuit smaller

data_wires = list(range(levels, 2 * levels))
pairs = [(i, j) for i in range(len(data_wires)) for j in range(i + 1, len(data_wires))]

gamma = np.array(0.8, requires_grad=True)
beta = np.array(0.4, requires_grad=True)

def maxcut_H(pairs, ws):
    coeffs = [-0.5] * len(pairs)
    obs = [qml.PauliZ(ws[i]) @ qml.PauliZ(ws[j]) for (i, j) in pairs]
    return qml.Hamiltonian(coeffs, obs)

cost_H = qml.Hamiltonian(
    [-0.5],
    [qml.PauliZ(data_wires[0]) @ qml.PauliZ(data_wires[1])]
)

def qaoa_layer():
    for (i, j) in pairs:
        qml.CNOT(wires=[data_wires[i], data_wires[j]])
        qml.RZ(2 * gamma, wires=data_wires[j])
        qml.CNOT(wires=[data_wires[i], data_wires[j]])
    for w in data_wires:
        qml.RX(2 * beta, wires=w)



In [18]:
#so now we build the circuit with da qaoa layer

dev = qml.device("lightning.qubit", wires=num_wires, shots=shots)

with qml.queuing.AnnotatedQueue() as q:
    def full_circuit():
        galton = qgb.galton_template(levels, bias=0.3, coherence=False)
        galton()
        qaoa_layer()
        return qml.expval(qml.PauliZ(data_wires[0]))

    full_circuit()

In [22]:
#okay now we cut the circuit

# Convert to tape
tape = qml.tape.QuantumScript.from_queue(q, shots=shots)
print("Measurements:", tape.measurements)
print("Measurement count:", len(tape.measurements))
print("Measurement types:", [type(m) for m in tape.measurements])

for i, op in enumerate(tape.operations):
    print(f"{i:02d} | {op.name:<15} | wires={op.wires} | type={type(op)}")

# Apply cut_circuit
cut_transform = lambda t: cut_circuit.transform(t, device_wires=dev.wires, cuts=cut_wires)
fragment_tapes, processing_fn = cut_transform(tape)

Measurements: [expval(Z(6))]
Measurement count: 1
Measurement types: [<class 'pennylane.measurements.expval.ExpectationMP'>]
00 | RX              | wires=Wires([0]) | type=<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RX'>
01 | PauliX          | wires=Wires([6]) | type=<class 'pennylane.ops.qubit.non_parametric_ops.PauliX'>
02 | CSWAP           | wires=Wires([0, 5, 6]) | type=<class 'pennylane.ops.op_math.controlled_ops.CSWAP'>
03 | CNOT            | wires=Wires([6, 0]) | type=<class 'pennylane.ops.op_math.controlled_ops.CNOT'>
04 | CSWAP           | wires=Wires([0, 6, 7]) | type=<class 'pennylane.ops.op_math.controlled_ops.CSWAP'>
05 | MidMeasureMP    | wires=Wires([0]) | type=<class 'pennylane.measurements.mid_measure.MidMeasureMP'>
06 | RX              | wires=Wires([0]) | type=<class 'pennylane.ops.qubit.parametric_ops_single_qubit.RX'>
07 | CSWAP           | wires=Wires([0, 4, 5]) | type=<class 'pennylane.ops.op_math.controlled_ops.CSWAP'>
08 | CNOT            | wires=Wi

ValueError: Only a single return type can be used for measurement nodes in graph_to_tape

In [ ]:
from pennylane import qcut

# Step 1: Get the transform function (instead of calling directly)
cut_transform = qcut.cut_circuit.transform

# Step 2: Apply it to the QNode
fragment_tapes, processing_fn = cut_transform(full_circuit)

# Step 3: Evaluate each tape individually
results = [qml.execute([tape], dev, None)[0] for tape in fragment_tapes]

# Step 4: Stitch the result
final_result = processing_fn(results)
print("⟨H⟩ after manual stitching:", final_result)


AttributeError: 'QNode' object has no attribute 'measurements'

In [6]:
opt    = qml.GradientDescentOptimizer(0.2)
params = qml.numpy.array([0.5, 0.3], requires_grad=True)   # [γ, β]

def cut_cost_fn(p):
    g, b = p
    gamma[...] = g;  beta[...] = b
    return stitch([f() for f in frags])

for it in range(15):
    params = opt.step(cut_cost_fn, params)
    if it % 5 == 0:
        print(f"Iter {it:02d}  cost = {cut_cost_fn(params):.6f}")

print("Optimised (γ, β):", params)
print("Final cut cost  :", cut_cost_fn(params))

ValueError: setting an array element with a sequence.

In [4]:
import pennylane as qml
from pennylane import qcut

#A properly working cut_circuit method

dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def full_circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    return qml.expval(qml.Z(0))  # Single measurement

# Use construct() to get the tape
tape = full_circuit.construct([], {})  # no parameters
print("Measurements:", tape.measurements)
print("Measurement count:", len(tape.measurements))

# Apply cut_circuit
cut_transform = lambda t: qcut.cut_circuit.transform(t, device_wires=dev.wires)
fragment_tapes, processing_fn = cut_transform(tape)


# Evaluate
results = [qml.execute([frag], dev, None)[0] for frag in fragment_tapes]
final_result = processing_fn(results)
print("⟨Z⟩ after cutting:", final_result)


Measurements: [expval(Z(0))]
Measurement count: 1
⟨Z⟩ after cutting: 0.0


In [52]:
@qml.qnode(dev)
def full_circuit():
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    return qml.expval(qml.Z(0))


In [ ]:


tape = make_tape(full_circuit)()


In [54]:
from pennylane import qcut

cut_transform = qcut.cut_circuit.transform
fragment_tapes, processing_fn = cut_transform(tape)


ValueError: The circuit cutting workflow only supports circuits with a single output measurement